## MSA

In [1]:
from Bio import AlignIO
from pathlib import Path
import subprocess
from Bio import SeqIO
import os
import re

In [2]:
# https://github.com/choderalab/asapdiscovery/blob/seq-alignment/asapdiscovery-seqalign/align_code.ipynb
def multi_seq_alignment(input_file, output_file):
    # Run MAFFT in command line
    cmd = f"mafft {input_file} > {output_file}"
    subprocess.run(cmd, shell=True)

    # Read in the alignment to 
    align = AlignIO.read(output_file, "fasta")
    return align, output_file

In [ ]:
local_path = Path('/home/pengs/fold_zika')
align, out_file = multi_seq_alignment(local_path/"clear_blank_seqs_found.txt", local_path/"output.fasta")


In [ ]:
fasta_sequences = list(SeqIO.parse(open(local_path/"output.fasta"),'fasta'))
for fasta in fasta_sequences:
    print(fasta.id)
    print(fasta.seq)

In [1]:
## Clean the sequences by removing '-' gaps
## Match the chains
## Remove the end of chain if not matched

In [ ]:
ref_file = list(SeqIO.parse(open(local_path/"zikv_ns2b3.fasta"),'fasta'))
for chain in ref_file:
    print(chain.id)
    print(chain.seq)

## This will a file where I desperately try to redock the ligands and documenting the command line progress to the best of my effort

### I will need to figure out a way to run the cross docking and specify things with the prepping process

## Reference:
### Somehow the asapdiscovery method of docking uses smile strings for the ligand input, would explore all poses, so may be good
### Probably use Cross-docking --> more underlying docking options from command line
### Cross-docking is moved into asapdiscovery-workflow in the remote main git

https://www.notion.so/asapdiscovery/HOWTO-Running-the-small-scale-docking-workflow-V2-44560183c5a7472c943a5b13fbce96b3
https://www.notion.so/asapdiscovery/HOWTO-Running-the-large-scale-docking-workflow-V2-32c3603915764ae5a85027e41d267ead
https://github.com/choderalab/asapdiscovery/blob/main/asapdiscovery-docking/asapdiscovery/docking/docking.py

asap-prep small-scale 
--target ZIKV-NS2B-NS3pro
--use-dask
--pdb-file my_structure.pdb

### Putting in the combined protein file because it seem to run and prep but not dock
asap-docking cross-docking --ligands /home/pengs/fold_zika/fragments/smiles.smi --pdb-file /home/pengs/fold_zika/combined/NP_722463.1/NP_722463.1_lig_0.pdb --target ZIKV-NS2B-NS3pro --cache-dir /home/pengs/fold_zika/temp_dir --use-dask --dask-failure-mode raise --dask-type lilac-gpu

outfile:
INFO     Selecting pairs for docking    cross_docking.py:167
INFO     Selected 0 pairs for docking   cross_docking.py:176

stderr:
ValueError: No docking results generated, exiting

### Was using LeaveSimilarOut selector, and seem to have some issues
### Changed to Pairwise selector
asap-docking cross-docking --ligands /home/pengs/fold_zika/fragments/lig_0.smi --pdb-file /home/pengs/fold_zika/combined/NP_722463.1/NP_722463.1_lig_0.pdb --target ZIKV-NS2B-NS3pro --cache-dir /home/pengs/fold_zika/temp_dir --use-dask --dask-failure-mode raise --structure-selector PairwiseSelector --output-dir /home/pengs/fold_zika

https://github.com/choderalab/asapdiscovery/blob/173047763dbbef4c9c42bbd3d17c8772a2272ee1/asapdiscovery-data/asapdiscovery/data/operators/selectors/selector_list.py

## What I am running right now
### Seem to have some issues running script since it deleted whatever is in the directory the script indicate cd to
### OUTPUT FOLDER will override everything in pre-existing folder, so better to have a new folder for each run
#BSUB -J FOLD
#BSUB -n 1
#BSUB -R rusage[mem=32]
#BSUB -q gpuqueue
#BSUB -gpu "num=1"
#BSUB -W 10:00
#BSUB -o log_files/dock_test.out
#BSUB -e log_files/dock_test.stderr
source ~/.bashrc
cd /home/pengs/fold_zika/test_dock
conda activate asapdiscovery

asap-docking cross-docking --ligands lig_0.smi --pdb-file NP_722463.1_lig_0.pdb --target ZIKV-NS2B-NS3pro --cache-dir /home/pengs/fold_zika/temp_dir --use-dask --dask-failure-mode raise --structure-selector PairwiseSelector --output-dir /home/pengs/fold_zika/test_dock

echo done

## Use   --use-only-cache to indicate using prepped structures

### Must have ligand or active ligand identified to prepare protein structure

## Change what I need in the git branch cross_dock on lilac

## Run cross-docking not from cli

In [ ]:
from asapdiscovery.docking.workflows.cross_docking import (
    CrossDockingWorkflowInputs,
    cross_docking_workflow,
)

In [ ]:
if input_json is not None:
    print("Loading inputs from json file.")
    ## Gotta write input_json file from this thing
    ## Gotta configure cross_dock again to so the inclusion of more options will work
    inputs = CrossDockingWorkflowInputs.from_json_file(input_json)

cross_docking_workflow(inputs)

## If I can't get cross-docking to work, just go back to openeye to run this
https://github.com/choderalab/asapdiscovery/blob/173047763dbbef4c9c42bbd3d17c8772a2272ee1/asapdiscovery-docking/asapdiscovery/docking/openeye.py#L112


In [1]:
from asapdiscovery.docking.openeye import POSIT_METHOD, POSIT_RELAX_MODE, POSITDocker

## After redocking
1. realign proteins
2. figure out if the ligand still occupy about the same space
   - might be good to calculate the ligand RMSD (https://github.com/choderalab/asapdiscovery/blob/173047763dbbef4c9c42bbd3d17c8772a2272ee1/asapdiscovery-docking/asapdiscovery/docking/analysis.py#L427)

In [ ]:
# Align the docked structure to the original zikv_ns2b3 structure
# OR maybe just use the just folded protein with the original ligand position combined to it
ref_path = Path('/home/pengs/fold_zika/orig_bind')

# Docked structure path
dock_path

In [ ]:
# Align based on the protein in each of the files

In [2]:
from asapdiscovery.docking.analysis import calculate_rmsd_openeye
from pathlib import Path

In [ ]:
# Compare against reference ligand (position of zikv ns2b3 ligand position)
# Separate out the ligand

# Docked ligand
